In [1]:
# ----------------------------
# Package Installation
# ----------------------------
# Install TensorFlow Hub for pre-trained models
!pip install tensorflow_hub --quiet  # Suppress installation logs

# Install headless OpenCV for image processing (no GUI dependencies)
!pip install opencv-python-headless --quiet

# ----------------------------
# Core Imports
# ----------------------------
# TensorFlow ecosystem
import tensorflow as tf                 # Main ML framework
import tensorflow_hub as hub            # Pre-trained model repository

# Computer Vision
import cv2                              # Image processing
from google.colab.patches import cv2_imshow  # Colab-compatible image display

# Data Handling
import numpy as np                      # Numerical operations
import json                             # JSON file processing
import os                               # File system operations

# Google Colab Integration
from google.colab import drive          # Mount Google Drive
from google.colab import files          # File upload/download

# Machine Learning Utilities
from sklearn.model_selection import train_test_split  # Data splitting
from sklearn.ensemble import RandomForestClassifier  # Our classifier
from sklearn.metrics import (          # Model evaluation
    accuracy_score,
    classification_report,
    confusion_matrix
)

# Visualization
import matplotlib.pyplot as plt         # Plotting and visualization
import seaborn as sns                   # Enhanced visualizations
plt.style.use('ggplot')                 # Professional plotting style

# ----------------------------
# Version Verification
# ----------------------------
print(f"TensorFlow Version: {tf.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")

# ----------------------------
# Configuration Constants
# ----------------------------
# Recommended to define these once at startup
MODEL_URL = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
INPUT_SIZE = 192  # Movenet input dimension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 552.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 25.8 MB/s eta 0:00:00
TensorFlow Version: 2.19.0
OpenCV Version: 4.11.0
NumPy Version: 2.0.2


## Img preprocessing


In [2]:
import os
from tqdm import tqdm
from google.colab import drive
from collections import defaultdict

# ----------------------------
# Google Drive Mounting
# ----------------------------
print("Mounting Google Drive...")
drive.mount('/content/drive')  # Requires authorization
print("Drive mounted successfully!\n")

# ----------------------------
# Configuration
# ----------------------------
# Define dataset path (modify as needed)
DATASET_PATH = "/content/drive/MyDrive/Yoga-Pose-Dataset-V1"
SUPPORTED_EXTENSIONS = ('.png', '.jpg', '.jpeg', '.webp')  # Add more if needed

# ----------------------------
# Dataset Analysis Function
# ----------------------------
def analyze_dataset(dataset_path):
    """
    Analyzes a yoga pose dataset directory structure and counts images per class.

    Args:
        dataset_path (str): Path to the root dataset directory

    Returns:
        tuple: (pose_counts, total_images, total_classes)
            - pose_counts: Dictionary of {pose_name: image_count}
            - total_images: Sum of all images
            - total_classes: Number of pose classes
    """
    pose_counts = defaultdict(int)
    problematic_folders = []

    # Validate dataset path exists
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Dataset path not found: {dataset_path}")

    # Process each pose folder with progress bar
    for pose_folder in tqdm(os.listdir(dataset_path), desc="Analyzing dataset"):
        folder_path = os.path.join(dataset_path, pose_folder)

        # Skip non-directory items
        if not os.path.isdir(folder_path):
            continue

        try:
            # Count valid image files
            valid_files = [
                f for f in os.listdir(folder_path)
                if f.lower().endswith(SUPPORTED_EXTENSIONS)
            ]
            pose_counts[pose_folder] = len(valid_files)

            # Warn if folder is empty
            if len(valid_files) == 0:
                problematic_folders.append(pose_folder)

        except Exception as e:
            print(f"\nError processing {pose_folder}: {str(e)}")
            problematic_folders.append(pose_folder)

    # Print warnings if any issues found
    if problematic_folders:
        print("\n⚠️ Warning: Issues found in these folders:")
        for folder in problematic_folders:
            print(f" - {folder}")

    return pose_counts, sum(pose_counts.values()), len(pose_counts)

# ----------------------------
# Main Execution
# ----------------------------
if __name__ == "__main__":
    try:
        # Run dataset analysis
        pose_counts, total_images, total_classes = analyze_dataset(DATASET_PATH)

        # Display results
        print("\n" + "="*40)
        print("Yoga Pose Dataset Summary".center(40))
        print("="*40)
        print(f"{'Pose Name':<25} | {'Count':>10}")
        print("-"*40)

        # Sort by count (descending)
        for pose, count in sorted(pose_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"{pose:<25} | {count:>10}")

        print("="*40)
        print(f"TOTAL POSES: {total_classes:>26}")
        print(f"TOTAL IMAGES: {total_images:>25}")
        print("="*40)

        # Basic statistics
        avg_images = total_images / total_classes if total_classes > 0 else 0
        print(f"\nAverage images per class: {avg_images:.1f}")
        print(f"Most images: {max(pose_counts.values())} ({max(pose_counts, key=pose_counts.get)})")
        print(f"Least images: {min(pose_counts.values())} ({min(pose_counts, key=pose_counts.get)})")

    except Exception as e:
        print(f"\n❌ Error in dataset analysis: {str(e)}")

Mounting Google Drive...
Mounted at /content/drive
Drive mounted successfully!



Analyzing dataset: 100%|██████████| 15/15 [00:01<00:00,  7.54it/s]


       Yoga Pose Dataset Summary        
Pose Name                 |      Count
----------------------------------------
Cow Pose                  |         87
Crane Pose                |         77
Upward Facing Dog         |         70
Low Lunge Pose            |         64
Shoulder Pressing Pose    |         61
Half Lord of the Fishes Pose |         60
Happy Baby Pose           |         59
Handstand Pose            |         59
Wild Thing Pose           |         54
Half Moon Pose            |         52
Sleeping Vishnu Pose      |         43
Half Frog Pose            |         40
Cat-Cow Stretch Pose      |         40
Frog Pose                 |         39
Eight Limbed Pose         |         33
TOTAL POSES:                         15
TOTAL IMAGES:                       838

Average images per class: 55.9
Most images: 87 (Cow Pose)
Least images: 33 (Eight Limbed Pose)


Image Augmentation

In [3]:
import os
from tqdm import tqdm  # Optional, for progress bar

# Path to your dataset
dataset_path = "/content/drive/MyDrive/Maulik DataSet"  # Change this to your folder path

# Dictionary to store counts
pose_counts = {}

# Get counts for each pose folder
for pose_folder in tqdm(os.listdir(dataset_path)):
    folder_path = os.path.join(dataset_path, pose_folder)

    # Skip if not a directory
    if not os.path.isdir(folder_path):
        continue

    # Count image files (supports .jpg, .png, .jpeg)
    image_count = 0
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_count += 1

    pose_counts[pose_folder] = image_count

# Display results
print("\nYoga Pose Image Counts:")
print("-" * 30)
for pose, count in pose_counts.items():
    print(f"{pose:20s}: {count} images")
print("-" * 30)
print(f"TOTAL POSES: {len(pose_counts)}")
print(f"TOTAL IMAGES: {sum(pose_counts.values())}")

100%|██████████| 15/15 [00:05<00:00,  2.55it/s]


Yoga Pose Image Counts:
------------------------------
Upward Facing Dog   : 474 images
Sleeping Vishnu Pose: 255 images
Half Moon Pose      : 330 images
Cow Pose            : 607 images
Crane Pose          : 527 images
Wild Thing Pose     : 343 images
Happy Baby Pose     : 383 images
Low Lunge Pose      : 429 images
Eight Limbed Pose   : 179 images
Handstand Pose      : 383 images
Half Frog Pose      : 231 images
Half Lord of the Fishes Pose: 391 images
Shoulder Pressing Pose: 469 images
Frog Pose           : 312 images
Cat-Cow Stretch Pose: 320 images
------------------------------
TOTAL POSES: 15
TOTAL IMAGES: 5633


EDA

In [1]:
!pip install opencv-python # Installing opencv-python in the current cell's runtime

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configuration
dataset_path = "/content/drive/MyDrive/Yoga-Pose-Dataset-V1"
output_path = "eda_results"
os.makedirs(output_path, exist_ok=True)

# 1. Data Distribution Analysis
pose_counts = {}
image_sizes = []
color_stats = []

for pose_folder in tqdm(os.listdir(dataset_path)):
    folder_path = os.path.join(dataset_path, pose_folder)
    if not os.path.isdir(folder_path):
        continue

    count = 0
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)

            # Collect image stats
            image_sizes.append(img.shape[:2])  # (height, width)

            # Collect color stats (BGR format)
            color_stats.append({
                'pose': pose_folder,
                'mean': img.mean(),
                'std': img.std()
            })
            count += 1

    pose_counts[pose_folder] = count

# Convert to DataFrames
df_counts = pd.DataFrame.from_dict(pose_counts, orient='index', columns=['count'])
df_sizes = pd.DataFrame(image_sizes, columns=['height', 'width'])
# Calculate aspect ratio and add it as a new column
df_sizes['aspect_ratio'] = df_sizes['width'] / df_sizes['height'] # Calculating aspect ratio
df_colors = pd.DataFrame(color_stats)


# 3. Generate Report
report = f"""
=== YOGA POSE DATASET EDA REPORT ===

1. Basic Statistics:
- Total poses: {len(df_counts)}
- Total images: {df_counts['count'].sum()}
- Avg images per pose: {df_counts['count'].mean():.1f}
- Most common pose: {df_counts.idxmax()[0]} ({df_counts.max()[0]} images)
- Rarest pose: {df_counts.idxmin()[0]} ({df_counts.min()[0]} images)

2. Image Characteristics:
- Average dimensions: {int(df_sizes['width'].mean())}x{int(df_sizes['height'].mean())} px
- Size variability: {df_sizes['width'].std():.1f} (width) ± {df_sizes['height'].std():.1f} (height)
- Most common aspect ratio: {df_sizes['aspect_ratio'].mode()[0]:.2f}

3. Color Analysis:
- Overall mean intensity: {df_colors['mean'].mean():.1f}
- Highest contrast pose: {df_colors.groupby('pose')['std'].mean().idxmax()}
- Darkest pose: {df_colors.groupby('pose')['mean'].mean().idxmin()}
"""

print(report)
with open(os.path.join(output_path, 'eda_report.txt'), 'w') as f:
    f.write(report)

100%|██████████| 15/15 [04:42<00:00, 18.82s/it]


=== YOGA POSE DATASET EDA REPORT ===

1. Basic Statistics:
- Total poses: 15
- Total images: 838
- Avg images per pose: 55.9
- Most common pose: Cow Pose (87 images)
- Rarest pose: Eight Limbed Pose (33 images)

2. Image Characteristics:
- Average dimensions: 427x342 px
- Size variability: 196.2 (width) ± 149.2 (height)
- Most common aspect ratio: 1.00

3. Color Analysis:
- Overall mean intensity: 159.8
- Highest contrast pose: Cat-Cow Stretch Pose
- Darkest pose: Frog Pose




<ipython-input-1-783d354774a4>:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  - Most common pose: {df_counts.idxmax()[0]} ({df_counts.max()[0]} images)
<ipython-input-1-783d354774a4>:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  - Rarest pose: {df_counts.idxmin()[0]} ({df_counts.min()[0]} images)


# Define Global variables

In [ ]:
# Load the MoveNet model from TensorFlow Hub
# Model: MoveNet SinglePose Lightning (fast, suitable for real-time applications)
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Define the mapping of keypoints to body parts
# These are the 17 keypoints detected by the MoveNet model in order
keypoint_names = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder',
                  'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
                  'left_knee', 'right_knee', 'left_ankle', 'right_ankle']

# Dictionary mapping keypoint indices to human-readable names
# Useful for labeling keypoints when visualizing or interpreting model output
keypoint_names_dir = {
        0: "Nose", 1: "Left Eye", 2: "Right Eye", 3: "Left Ear", 4: "Right Ear",
        5: "Left Shoulder", 6: "Right Shoulder", 7: "Left Elbow", 8: "Right Elbow",
        9: "Left Wrist", 10: "Right Wrist", 11: "Left Hip", 12: "Right Hip",
        13: "Left Knee", 14: "Right Knee", 15: "Left Ankle", 16: "Right Ankle"
    }

# Define the connections between keypoints for drawing pose skeleton
# Each tuple represents a pair of keypoints to be connected by a line
# Used in visualization to illustrate body posture
connections = [(0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6), (5, 7), (7, 9), (6, 8), (8, 10),
               (5, 6), (5, 11), (6, 12), (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)]


# Convert Image to Keypoints

In [ ]:
def detect_pose_static(image_path):
    """
    Detects human pose keypoints in a static image using the MoveNet model.

    Parameters:
        image_path (str): Path to the image file.

    Returns:
        numpy.ndarray: A NumPy array containing the detected keypoints.
                       Shape: [1, 1, 17, 3] where:
                           - 1st dim = batch size (always 1),
                           - 2nd dim = person count (always 1 for single-pose model),
                           - 3rd dim = 17 keypoints,
                           - 4th dim = [y, x, confidence] for each keypoint.
    """
    # Read the image from the given file path
    image = cv2.imread(image_path)

    # Convert the image from BGR (OpenCV default) to RGB (TensorFlow expects RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize the image to 192x192 with padding (required input size for MoveNet)
    # Add a batch dimension (axis=0) since the model expects a batch of images
    image_resized = tf.image.resize_with_pad(tf.expand_dims(image_rgb, axis=0), 192, 192)

    # Convert the image to NumPy array with int32 type as expected by the model
    image_np = image_resized.numpy().astype(np.int32)

    # Run the MoveNet model and get the keypoints
    # 'serving_default' is the default signature for inference
    outputs = movenet.signatures["serving_default"](tf.constant(image_np))

    # Extract the keypoints from the model output
    keypoints = outputs['output_0'].numpy()

    # Return the keypoints (shape: [1, 1, 17, 3] => batch, person, keypoints, [y, x, confidence])
    return keypoints


# Calculate Angle Difference

In [ ]:
def calculate_angle(a, b, c):
    """
    Calculates the angle (in degrees) formed at point 'b' by the line segments ab and bc.

    Parameters:
        a (list or np.array): Coordinates of the first point (e.g., [x, y]).
        b (list or np.array): Coordinates of the middle joint (angle vertex).
        c (list or np.array): Coordinates of the third point (e.g., [x, y]).

    Returns:
        float: The angle in degrees between the line segments ab and bc.
    """

    # Convert input points to NumPy arrays for vector operations
    a = np.array(a)  # First joint
    b = np.array(b)  # Middle joint (vertex)
    c = np.array(c)  # End joint

    # Vectors from the middle joint to the other two
    ba = a - b
    bc = c - b

    # Compute the cosine of the angle using the dot product formula
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    # Clip cosine to the valid domain of arccos to avoid numerical issues
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    # Convert the angle from radians to degrees
    return np.degrees(angle)


In [ ]:
# Function to check if an angle is within an acceptable range
def is_within_tolerance(expected, actual, tolerance):
    """
    Checks whether the actual angle is within the acceptable tolerance range of the expected angle.

    Parameters:
        expected (float): The target angle in degrees.
        actual (float): The detected angle in degrees.
        tolerance (float): The allowed deviation from the expected angle.

    Returns:
        bool: True if actual is within ±tolerance of expected, otherwise False.
    """
    return abs(expected - actual) <= tolerance


# Tolerance levels based on difficulty (in degrees)
# Higher tolerance for beginners, stricter for advanced users
tolerance_levels = {
    "Beginner": 50,       # More forgiving (±50°)
    "Intermediate": 15,   # Moderate strictness (±15°)
    "Advanced": 10        # Strictest (±10°)
}


# Default tolerance range per joint for "Advanced" level
# These values reflect how precise a joint angle should be
default_tolerance_ranges = {
    "Left Elbow": 10, "Right Elbow": 10,
    "Left Knee": 15, "Right Knee": 15,
    "Left Shoulder": 12, "Right Shoulder": 12,
    "Left Hip": 18, "Right Hip": 18
}


# Function to update joint tolerance values based on difficulty level
def adjust_tolerance_levels(level="Advanced"):
    """
    Adjusts the tolerance values for joint angles based on the user's skill level.

    Parameters:
        level (str): Difficulty level ("Beginner", "Intermediate", "Advanced").

    Returns:
        dict: A dictionary with updated tolerance per joint, scaled according to difficulty.
    """
    base_tolerance = tolerance_levels.get(level, 10)  # Default to Advanced if invalid level
    scale_factor = base_tolerance / 10  # Scale based on Advanced baseline (10)

    # Scale each joint's default tolerance by the calculated factor
    return {joint: int(value * scale_factor) for joint, value in default_tolerance_ranges.items()}



In [ ]:
def provide_correction_feedback(detected_keypoints, reference_keypoints, predicted_pose_name, skill_level="Intermediate"):
    """
    Provides feedback on pose accuracy by comparing detected keypoint angles to reference keypoints.

    Parameters:
        detected_keypoints (list): List of 17 detected keypoints from the user [ [x, y], ... ].
        reference_keypoints (list): List of reference pose data dicts. Each dict should have:
                                    - "pose": pose name (str)
                                    - "keypoints": list with shape [1, 1, 17, 3]
        predicted_pose_name (str): The pose name predicted for the user's pose.
        skill_level (str): User's skill level ("Beginner", "Intermediate", "Advanced").

    Returns:
        list: Feedback strings indicating corrections, or confirmation if pose is correct.
    """

    feedback = []

    # Critical joints represented as tuples of (start, middle, end) indices
    critical_joints = [(5, 7, 9),   # Left arm
                       (6, 8, 10),  # Right arm
                       (11, 13, 15),  # Left leg
                       (12, 14, 16)]  # Right leg

    # Get tolerance levels based on skill level
    default_tolerance = adjust_tolerance_levels(skill_level)

    # Filter reference keypoints for the predicted pose
    matching_references = [ref for ref in reference_keypoints if ref["pose"] == predicted_pose_name]

    # Extract all keypoints from the matching references
    all_keypoints = [ref["keypoints"] for ref in matching_references]

    if not all_keypoints:
        return ["Error: No reference keypoints found for the predicted pose."]

    # Validate detected keypoints
    if len(detected_keypoints) < 17:
        return ["Error: Invalid keypoint data."]

    for joints in critical_joints:
        try:
            # Compute angle from detected user pose
            detected_angle = calculate_angle(
                detected_keypoints[joints[0]],
                detected_keypoints[joints[1]],
                detected_keypoints[joints[2]]
            )

            # Compute average angle from all reference keypoints
            reference_angles = []
            for ref in all_keypoints:
                if len(ref[0][0]) < 17:
                    return ["Error: Invalid reference keypoint data."]
                ref_kps = ref[0][0]
                angle = calculate_angle(ref_kps[joints[0]], ref_kps[joints[1]], ref_kps[joints[2]])
                reference_angles.append(angle)

            reference_angle = sum(reference_angles) / len(reference_angles)

            # Get the joint name and its specific tolerance
            joint_name = keypoint_names_dir[joints[1]]
            tolerance = default_tolerance.get(joint_name, 10)  # fallback tolerance

            # Compare angles
            if not is_within_tolerance(reference_angle, detected_angle, tolerance):
                feedback.append(
                    f"Adjust angle at {joint_name}: Expected {reference_angle:.2f}°, got {detected_angle:.2f}° (Tolerance: ±{tolerance}°)."
                )

        except (IndexError, KeyError, TypeError) as e:
            return [f"Error: Missing or incorrect keypoints for joints {joints}. Exception: {str(e)}"]

    return feedback if feedback else ["Pose is correct!"]



In [ ]:
# Function to determine color based on deviation from expected angle
def get_color_based_on_tolerance(expected, actual, tolerance):
    """
    Determines a color (BGR format) based on how close the actual angle is to the expected angle.

    Parameters:
        expected (float): The reference or expected angle.
        actual (float): The detected angle from the user's pose.
        tolerance (float): The acceptable deviation from the expected angle.

    Returns:
        tuple: A BGR color tuple for OpenCV drawing:
               - Green (0, 255, 0) if within tolerance
               - Yellow (0, 255, 255) if slightly off (<= 1.5x tolerance)
               - Red (0, 0, 255) if significantly off (> 1.5x tolerance)
    """
    deviation = abs(expected - actual)

    if deviation <= tolerance:
        return (0, 255, 0)  # Green = Correct
    elif deviation <= tolerance * 1.5:
        return (0, 255, 255)  # Yellow = Needs attention
    else:
        return (0, 0, 255)  # Red = Needs correction


# Draw Keypoint in Upload Image

In [ ]:
def visualize_pose_static(image_path, keypoints, reference_keypoints=None, predicted_pose_name="", skill_level="Intermediate"):
    """
    Visualizes the detected pose on the given static image and provides correction feedback if reference keypoints are provided.

    Parameters:
        image_path (str): Path to the image file to be processed.
        keypoints (list or np.array): Detected keypoints in the format [ [x, y], ... ].
        reference_keypoints (list, optional): Reference keypoints for comparison to provide feedback on pose correctness.
        predicted_pose_name (str, optional): Name of the predicted pose to match with reference keypoints.
        skill_level (str, optional): Skill level for adjusting tolerance in corrections ("Beginner", "Intermediate", "Advanced").

    Returns:
        None: The function will display the image with drawn keypoints and connections, and print correction feedback if necessary.
    """

    # Read the image
    image = cv2.imread(image_path)
    keypoints = np.array(keypoints)

    # Adjust keypoints shape based on input format
    if len(keypoints.shape) == 4:
        keypoints = keypoints[0, 0]  # Extract if shape is (1,1,17,3)
    elif len(keypoints.shape) == 3:
        keypoints = keypoints[0]  # Extract if shape is (1,17,3)

    # Replace NaN values with zeros
    keypoints = np.nan_to_num(keypoints)

    height, width, _ = image.shape

    # Default color for keypoints if no feedback is provided
    default_color = (0, 255, 0)  # Green

    # Initialize feedback for pose correction
    feedback = []
    tolerance = 10  # Default angle tolerance
    default_tolerance = adjust_tolerance_levels(skill_level)  # Adjust based on skill level

    # If reference keypoints are provided, calculate correction feedback
    if reference_keypoints is not None:
        feedback = provide_correction_feedback(keypoints, reference_keypoints, predicted_pose_name, skill_level)

    # Draw keypoints
    for i, kp in enumerate(keypoints):
        x = int(kp[1] * width)
        y = int(kp[0] * height)

        # Set color based on feedback, default to green
        color = default_color

        if reference_keypoints is not None:
            # Check feedback and adjust color for the keypoint
            for msg in feedback:
                if keypoint_names_dir[i] in msg:
                    expected_angle = float(msg.split("Expected")[1].split("°")[0].strip())
                    actual_angle = float(msg.split("got")[1].split("°")[0].strip())
                    tolerance = default_tolerance.get(keypoint_names_dir[i], 10)
                    color = get_color_based_on_tolerance(expected_angle, actual_angle, tolerance)

        # Draw the keypoint as a circle
        cv2.circle(image, (x, y), 12, color, -1)

    # Draw connections between keypoints (limbs and torso)
    connections = [
        (5, 7), (7, 9), (6, 8), (8, 10),  # Arms
        (11, 13), (13, 15), (12, 14), (14, 16),  # Legs
        (5, 6), (11, 12), (5, 11), (6, 12)  # Torso
    ]

    for connection in connections:
        idx1, idx2 = connection
        x1, y1 = int(keypoints[idx1, 1] * width), int(keypoints[idx1, 0] * height)
        x2, y2 = int(keypoints[idx2, 1] * width), int(keypoints[idx2, 0] * height)

        # Set line color based on feedback
        line_color = default_color

        if reference_keypoints is not None:
            for msg in feedback:
                if keypoint_names_dir[idx1] in msg or keypoint_names_dir[idx2] in msg:
                    expected_angle = float(msg.split("Expected")[1].split("°")[0].strip())
                    actual_angle = float(msg.split("got")[1].split("°")[0].strip())
                    tolerance = default_tolerance.get(keypoint_names_dir[idx1], 10)
                    line_color = get_color_based_on_tolerance(expected_angle, actual_angle, tolerance)

        # Draw the line between keypoints
        cv2.line(image, (x1, y1), (x2, y2), line_color, 4)

    # Display the image with drawn keypoints and connections
    cv2_imshow(image)  # Display in Colab

    # Print corrections if necessary
    if feedback:
        print("\nCorrections Needed:")
        for msg in feedback:
            print(msg)


# Get Dataset and save Keypoint Data

In [ ]:
# Mount Google Drive to access the dataset
drive.mount('/content/drive')

# Define the path to the dataset folder in Google Drive
dataset_path = "/content/drive/MyDrive/Maulik DataSet"

# Initialize an empty list to store pose data (class and keypoints)
pose_data = []

# Loop through each pose class folder in the dataset directory
for pose_class in os.listdir(dataset_path):
    # Get the full path to the current pose class folder
    pose_folder = os.path.join(dataset_path, pose_class)

    # Skip if the item is not a directory
    if not os.path.isdir(pose_folder):
        continue

    # Process each image file in the current pose class folder
    for img_file in os.listdir(pose_folder):
        # Get the full path to the current image
        img_path = os.path.join(pose_folder, img_file)

        # Detect pose keypoints from the static image
        keypoints = detect_pose_static(img_path)

        # Append the pose class and corresponding keypoints to the list
        pose_data.append({"pose": pose_class, "keypoints": keypoints.tolist()})

# Define the output path for the JSON file containing all keypoints
json_path = "/content/yoga_keypoints.json"

# Save the collected pose data to a JSON file
with open(json_path, "w") as f:
    json.dump(pose_data, f)

# Print confirmation that keypoints were saved
print(f"Keypoints saved to {json_path}")

# Verify the saved data by loading it back
with open("/content/yoga_keypoints.json", "r") as f:
    data = json.load(f)
    print(f"Loaded {len(data)} keypoints from {json_path}")

# Build the Model to Predict

In [ ]:
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load keypoint data from JSON file
with open("/content/yoga_keypoints.json", "r") as f:
    data = json.load(f)

# Initialize empty lists for features (X) and labels (y)
X = []
y = []

# Process each item in the loaded data
for item in data:
    # Convert keypoints to numpy array and flatten into 1D vector
    X.append(np.array(item["keypoints"]).flatten())  # Flatten keypoints
    # Add corresponding pose label
    y.append(item["pose"])

# Split data into training and testing sets (80% train, 20% test)
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Initialize Random Forest classifier with 100 decision trees
clf = RandomForestClassifier(n_estimators=100)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Evaluation Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Get unique pose classes from test labels for consistent ordering
pose_classes = np.unique(y_test)

# Generate predictions on the test set using trained classifier
y_pred = clf.predict(X_test)

# Print detailed classification report
print("Classification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=pose_classes  # Use actual class names for readability
))

# Generate and visualize confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a heatmap visualization of the confusion matrix
plt.figure(figsize=(10, 8))  # Set figure size for readability
sns.heatmap(
    cm,
    annot=True,            # Display values in each cell
    fmt='d',               # Format as integers
    cmap='Blues',          # Blue color gradient
    xticklabels=pose_classes,  # Use class names for x-axis
    yticklabels=pose_classes   # Use class names for y-axis
)
plt.title('Confusion Matrix')  # Add title
plt.xlabel('Predicted')       # Label x-axis
plt.ylabel('Actual')          # Label y-axis
plt.show()                   # Display the plot

In [ ]:
# Import required libraries
from sklearn.ensemble import RandomForestClassifier  # For Random Forest implementation
from sklearn.metrics import accuracy_score          # For calculating accuracy metrics
import matplotlib.pyplot as plt                    # For visualization

# Define range of tree counts to evaluate (from 1 to 100 trees, in increments of 10)
n_estimators = list(range(1, 101, 10))  # [1, 11, 21, ..., 91]

# Initialize lists to store performance metrics
train_accuracies = []  # Will store training accuracy for each tree count
test_accuracies = []   # Will store testing accuracy for each tree count
train_errors = []      # Will store training error (1 - accuracy) for each tree count
test_errors = []       # Will store testing error (1 - accuracy) for each tree count

# Iterate through each tree count configuration
for n in n_estimators:
    # Initialize Random Forest classifier with current parameters
    clf = RandomForestClassifier(
        n_estimators=n,               # Current number of trees being evaluated
        max_depth=10,                 # Prevent overfitting by limiting tree depth
        min_samples_split=5,          # Minimum samples required to split a node
        min_samples_leaf=3,           # Minimum samples required at each leaf node
        max_features='sqrt',          # Number of features to consider at each split (√n_features)
        random_state=42               # Seed for reproducibility
    )

    # Train the model on training data
    clf.fit(X_train, y_train)

    # Calculate training and testing accuracy
    train_acc = accuracy_score(y_train, clf.predict(X_train))  # Accuracy on training set
    test_acc = accuracy_score(y_test, clf.predict(X_test))     # Accuracy on test set

    # Store metrics
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    train_errors.append(1 - train_acc)  # Convert accuracy to error rate
    test_errors.append(1 - test_acc)    # Convert accuracy to error rate

    # Print current iteration results in formatted output
    print(f"Trees: {n:3d} | Train Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")

# ----------------------------
# Visualization 1: Accuracy Curve
# ----------------------------
plt.figure(figsize=(8, 6))  # Set figure size

# Plot training accuracy (blue line)
plt.plot(n_estimators, train_accuracies,
         label="Training Accuracy",  # Legend label
         linewidth=2.5,              # Line thickness
         color='#1f77b4')           # Matplotlib default blue

# Plot testing accuracy (green line)
plt.plot(n_estimators, test_accuracies,
         label="Test Accuracy",
         linewidth=2.5,
         color='#2ca02c')           # Matplotlib default green

# Chart styling
plt.xlabel("Number of Trees", fontsize=12)      # X-axis label
plt.ylabel("Accuracy", fontsize=12)             # Y-axis label
plt.title("Random Forest Accuracy Curve", fontsize=14, pad=15)  # Title with padding
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Dashed grid lines
plt.legend(loc='lower right', framealpha=1)     # Legend in lower right with solid background

# Set dynamic y-axis limits based on min/max accuracy values with 2% padding
plt.ylim(min(min(train_accuracies), min(test_accuracies)) - 0.02,
         max(max(train_accuracies), max(test_accuracies)) + 0.02)

plt.tight_layout()  # Adjust layout to prevent label clipping
plt.show()          # Display the plot

# ----------------------------
# Visualization 2: Loss Curve
# ----------------------------
plt.figure(figsize=(8, 6))  # Set figure size

# Plot training error (blue line)
plt.plot(n_estimators, train_errors,
         label="Training Loss",
         color='#1f77b4')  # Consistent color scheme

# Plot testing error (green line)
plt.plot(n_estimators, test_errors,
         label="Test Loss",
         color='#2ca02c')

# Chart styling
plt.xlabel("Number of Trees", fontsize=12)
plt.ylabel("Loss (1 - Accuracy)", fontsize=12)
plt.title("Random Forest Loss Curve", fontsize=14)
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)  # Same grid style as above
plt.legend(fontsize=12)  # Show legend with consistent font size
plt.tight_layout()
plt.show()

In [ ]:
def predict_pose(image_path):
    """
    Predicts the yoga pose from an input image using a trained classifier.

    Args:
        image_path (str): Path to the input image file

    Returns:
        tuple:
            - str: Predicted pose class name
            - numpy.ndarray: Detected keypoints array

    Process Flow:
        1. Detects body keypoints from the input image
        2. Flattens the keypoints for classifier input
        3. Makes prediction using pre-trained Random Forest
        4. Returns both prediction and raw keypoints
    """

    # Step 1: Detect body keypoints from image
    # Uses your custom pose detection function
    keypoints = detect_pose_static(image_path)

    # Step 2: Prepare keypoints for classifier
    # Flatten the keypoints array and reshape for sklearn's predict()
    # reshape(1, -1) converts it to 2D array with 1 sample (required format)
    keypoints_flat = keypoints.flatten().reshape(1, -1)

    # Step 3: Make prediction using trained classifier
    # clf should be your pre-trained RandomForestClassifier
    # [0] gets the first (and only) prediction from the 1D result array
    prediction = clf.predict(keypoints_flat)[0]

    # Step 4: Return both prediction and original keypoints
    # Returning keypoints allows for visualization or further processing
    return prediction, keypoints

# Getting Feedback From Json File

In [ ]:
# Install Google API client library (only needed first time)
!pip install google-api-python-client

from google.colab import auth
from googleapiclient.discovery import build
import json

# ----------------------------
# Google Drive Authentication
# ----------------------------
def authenticate_drive():
    """Authenticates the user and sets up Google Drive API service."""
    auth.authenticate_user()  # Triggers Google authentication flow
    return build('drive', 'v3')  # Returns Drive API service instance

# Initialize Drive service
drive_service = authenticate_drive()

# ----------------------------
# Configuration
# ----------------------------
# File ID extracted from Google Drive shareable link
POSE_FEEDBACK_FILE_ID = "1F7NEMAqa4a63P53JPRTmaYeI1hkTFvWQ"

# ----------------------------
# Data Loading Functions
# ----------------------------
def load_feedback_file(service, file_id):
    """
    Loads JSON feedback file from Google Drive.

    Args:
        service: Authenticated Drive API service
        file_id: Google Drive file ID

    Returns:
        dict: Parsed JSON content
    """
    try:
        request = service.files().get_media(fileId=file_id)
        json_content = request.execute().decode("utf-8")
        return json.loads(json_content)
    except Exception as e:
        print(f"Error loading feedback file: {str(e)}")
        return {}

# Load pose feedback data
pose_feedback = load_feedback_file(drive_service, POSE_FEEDBACK_FILE_ID)

# ----------------------------
# Feedback Retrieval
# ----------------------------
def get_pose_feedback(pose_name):
    """
    Retrieves feedback for a specific yoga pose.

    Args:
        pose_name (str): Name of the yoga pose

    Returns:
        str: Formatted feedback message or default not found message

    Example:
        >>> get_pose_feedback("Tree Pose")
        "For Tree Pose: Keep your spine straight and focus on a fixed point..."
    """
    feedback = pose_feedback.get(pose_name)

    if feedback:
        return f"For {pose_name}:\n{feedback}"
    else:
        return f"No specific feedback available for {pose_name}. Focus on proper alignment and breathing."


# Testing by Image Upload

In [ ]:
import json
from google.colab import files
import os

# ----------------------------
# Image Upload and Validation
# ----------------------------
print("Please upload a yoga pose image for analysis...")
uploaded = files.upload()  # Triggers file upload dialog

# Get the first uploaded filename
test_image = list(uploaded.keys())[0]
print(f"\nUploaded image: {test_image}")

# Validate the file exists
if not os.path.exists(test_image):
    print(f"\nError: The uploaded image '{test_image}' could not be found.")
    exit()

# ----------------------------
# Pose Prediction
# ----------------------------
# Get reference keypoints from your dataset
reference_keypoints = pose_data

# Make prediction
try:
    predicted_pose, keypoints = predict_pose(test_image)
    print(f"\nModel prediction: {predicted_pose}")

    # Visualize the pose with reference comparison
    visualize_pose_static(
        test_image,
        keypoints,
        reference_keypoints,
        predicted_pose,
        'Beginner'  # Default level
    )

except Exception as e:
    print(f"\nError during pose prediction: {str(e)}")
    exit()

# ----------------------------
# Feedback Retrieval
# ----------------------------
try:
    feedback = get_pose_feedback(predicted_pose)

    print("\n" + "="*50)  # Visual separator

    # Enhanced feedback display
    if isinstance(feedback, dict):
        print("\nPOSE ANALYSIS REPORT:")
        print(f"\nPose: {predicted_pose}")

        print("\nADVANTAGES:")
        print("- " + "\n- ".join(feedback.get("advantages", ["No specific advantages noted"])))

        print("\nRISKS:")
        print("- " + "\n- ".join(feedback.get("risks", ["No specific risks noted"])))

        if "tips" in feedback:
            print("\nPRO TIPS:")
            print("- " + "\n- ".join(feedback["tips"]))
    else:
        print(feedback)  # Fallback for string feedback

except Exception as e:
    print(f"\nError retrieving feedback: {str(e)}")

In [ ]:
import pickle
with open('yoga_pose_detection.pickle','wb') as f:
  pickle.dump(clf,f)